# Tree Class & Methods

##### Relate Trees Algorithm
Look at the next tree:
    1) Go through its edges and any edge labeled -1 is changed to edgeCount and edgeCount is increased.
    2) Search all trees after it in the list for equivalent edges.  Only edges which are labelled -1 might be equivalent.

In [141]:
import pandas as pd

def relateTrees(treelist):
    '''Version 3.0! Faster, less redundancy, and minimalized to a great degree.
    Checks all trees in treelist for equivalent edges and relabels all edges
    according to their unique or equivalent edge labels.  Assumes treelist is
    a list of trees. Complexity: Worst case you have m trees and n splits per
    tree which are all unique and must all be checked.  So, this could take
    n*(m-1)*n+n*(m-2)*n+... = n^2*(m-1+m-2+...+2+1) = n^2 m(m+1)/2.  In the
    best case scenario, All trees have the same splits and it takes n*(m-1).'''
    #Clear edge labels.
    for i in range(len(treelist)): #Go through each tree in order.
        for j in range(len(treelist[i].df["Edge"])): #Go through each edge.
            treelist[i].df.at[j,"Edge"]=-1
    
    #Begin relabeling equivalent and unique edges.
    edgeCount = 0
    for i in range(len(treelist)): #Go through each tree in order.
        for j in range(len(treelist[i].df["Edge"])): #Look at each edge label.
            # Check if edge j in tree i has label -1.  If so, relabel and check following trees.
            if treelist[i].df.at[j,"Edge"] == -1:
                treelist[i].df.at[j,"Edge"] = edgeCount
                edgeCount += 1
                #check following trees for equivalent edges.
                for k in range(i+1,len(treelist)): #Go through each tree after tree i.
                    for l in range(len(treelist[k].df["Edge"])): #Look at each Edge/Split.
                        if treelist[k].df.at[l,"Edge"] == -1: # See if edge is unlabled.
                            if spAreEquiv(treelist[i].df.at[j,"Splits"],treelist[k].df.at[l,"Splits"]): #If so, check if its equivalent.
                                treelist[k].df.at[l,"Edge"] = treelist[i].df.at[j,"Edge"]                 
    return

def spAreEquiv(split1,split2):
    '''Checks if this split1 is equivalent to split2. Splits
        are equivalent if their partitions of the leaves are equal as sets.
        Since splits are sorted lists, it checks element by element.'''
    #Check that splits come from graphs of equal leaf count.
    if len(split1.lsplit()+split1.rsplit()) != len(split2.lsplit()+split2.rsplit()):
        return "Splits are from trees of different leaf counts."
    if len(split1.lsplit())!=len(split2.lsplit()):
         return False
    else:
        for i in range(len(split1.lsplit())):
            if split1.lsplit()[i] != split2.lsplit()[i]:
                return False
        for i in range(len(split1.rsplit())):
            if split1.rsplit()[i] != split2.rsplit()[i]:
                return False
        return True


class Split():
    '''A pair of lists containing sets of leaves separated by removing an edge.'''
    def __init__(self,split1,split2,edgename):
        #sorts out language for splits.  The shorter is called the "left" split.
        if len(split1) <= len(split2):
            self.leftsplit = split1 #A shorter list of leaves in the tree-component.
            self.rightsplit = split2 #A longer list of leaves in the tree-component.
        else:
            self.leftsplit = split2 #A list of leaves in the tree's left-component.
            self.rightsplit = split1 #A list of leaves in the tree's right-component.
        self.edge = edgename
    
    def __str__(self):
        return str(self.leftsplit)+"|"+str(self.rightsplit)
    
    def lsplit(self):
        return self.leftsplit
    
    def rsplit(self):
        return self.rightsplit
    
    def setEdgeName(self,edgename):
        self.edge = edgename
        
    def edgeName(self):
        return self.edge
        

class Tree():
    '''Trees are described here as interior edge names, lengths,
    and the splits of leaves.  Each tree has n+1 leaves (n leaves + 1 root), 
    Initialize a tree by giving a list of edges, their lengths, and their
    splits, respectively.  The initializer will create a dataframe
    from these data. DF from dictionary {"Edges:[...], edgelength:[...],
    lsplit:[[]...], rsplit:[[]...]}
    for each interior edge in the tree.
    -Edgelength should be a list of float values.
    -lsplit and rsplit should be lists of lists.
    
    Edge names are automatically generated in relation to self. Use the
    method relateTrees(treelist) to automatically generate edge names 
    relative to other trees.'''
    
    def __init__(self,lengths,leftsplits,rightsplits):
        #Generates the splits from the lists given.  Sorts the splits in case the 
        #user did not enter them in accordingly.
        edges = [-1 for length in lengths]
        splits = []
        for i in range(len(leftsplits)):
            leftsplits[i].sort()
            rightsplits[i].sort()
            splits.append(Split(leftsplits[i],rightsplits[i],edges[i]))
        
        edgeDataDict={"Edge":edges, 
         "Lengths":lengths,
        "Splits":splits}
        self.df = pd.DataFrame(edgeDataDict)
        
        relateTrees([self, self])
        
    def __str__(self):
        return str(self.df)      
        
    

In [142]:
Tree1 = Tree([1.0,2.0,10.0,9,1],
    [[1,2],[0,1,2],[3,5],[0,1,2,3],[0,2,3,5]], [[0,3,4,5],[3,4,5],[0,1,2,4],[4,5],[1,4]])
Tree2 = Tree([1.0,2.0,5.0,8,1],
    [[2,1,4],[1,3],[0,1,2,3],[1,2],[0,2,3,4]], [[5,0,3],[0,2,5,4],[4,5],[0,3,4,5],[1,5]])
Tree3 = Tree([1.0,2.0,5.0,8,1],
    [[1,2],[2,1,0],[1,5],[0,1,2,3],[0,2,3,4]], [[0,3,4,5],[5,4,3],[0,2,3,4],[4,5],[1,5]])
Tree4 = Tree([1.0,2.0,5.0,8,1],
    [[1,2],[1,5],[3,5],[1,2,3],[0,2,3,4]], [[0,3,4,5],[0,2,3,4],[0,1,2,4],[0,4,5],[1,5]])
Tree5 = Tree([1.0,2.0,10.0,9,1],
    [[1,2],[0,1,2],[3,5],[0,1,2,5],[0,2,3,4]], [[0,3,4,5],[3,4,5],[0,1,2,4],[4,3],[1,5]])

print(Tree2)

   Edge  Lengths               Splits
0     0      1.0  [1, 2, 4]|[0, 3, 5]
1     1      2.0  [1, 3]|[0, 2, 4, 5]
2     2      5.0  [4, 5]|[0, 1, 2, 3]
3     3      8.0  [1, 2]|[0, 3, 4, 5]
4     4      1.0  [1, 5]|[0, 2, 3, 4]


In [143]:
relateTrees([Tree1,Tree2,Tree3,Tree4,Tree5])
print(Tree1.df)
print(Tree2.df)
print(Tree3.df)
print(Tree4.df)
print(Tree5.df)

   Edge  Lengths               Splits
0     0      1.0  [1, 2]|[0, 3, 4, 5]
1     1      2.0  [0, 1, 2]|[3, 4, 5]
2     2     10.0  [3, 5]|[0, 1, 2, 4]
3     3      9.0  [4, 5]|[0, 1, 2, 3]
4     4      1.0  [1, 4]|[0, 2, 3, 5]
   Edge  Lengths               Splits
0     5      1.0  [1, 2, 4]|[0, 3, 5]
1     6      2.0  [1, 3]|[0, 2, 4, 5]
2     3      5.0  [4, 5]|[0, 1, 2, 3]
3     0      8.0  [1, 2]|[0, 3, 4, 5]
4     7      1.0  [1, 5]|[0, 2, 3, 4]
   Edge  Lengths               Splits
0     0      1.0  [1, 2]|[0, 3, 4, 5]
1     1      2.0  [0, 1, 2]|[3, 4, 5]
2     7      5.0  [1, 5]|[0, 2, 3, 4]
3     3      8.0  [4, 5]|[0, 1, 2, 3]
4     7      1.0  [1, 5]|[0, 2, 3, 4]
   Edge  Lengths               Splits
0     0      1.0  [1, 2]|[0, 3, 4, 5]
1     7      2.0  [1, 5]|[0, 2, 3, 4]
2     2      5.0  [3, 5]|[0, 1, 2, 4]
3     8      8.0  [1, 2, 3]|[0, 4, 5]
4     7      1.0  [1, 5]|[0, 2, 3, 4]
   Edge  Lengths               Splits
0     0      1.0  [1, 2]|[0, 3, 4, 5]
1     1     